In [ ]:
pwd

'/content/drive/MyDrive'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd drive/MyDrive

[Errno 2] No such file or directory: 'drive/MyDrive'
/content/drive/MyDrive


### .mat -> .npy

In [ ]:
import os
import numpy as np
import matplotlib.pylab as plt
import scipy.io

mat_dir_path = os.path.join(os.getcwd(), 'DEAP','data_preprocessed_matlab')
mat_file_list = os.listdir(mat_dir_path)
print(mat_file_list)

['s01.mat', 's15.mat', 's29.mat', 's28.mat', 's14.mat', 's16.mat', 's02.mat', 's03.mat', 's17.mat', 's13.mat', 's07.mat', 's06.mat', 's12.mat', 's04.mat', 's10.mat', 's11.mat', 's05.mat', 's20.mat', 's08.mat', 's09.mat', 's21.mat', 's23.mat', 's22.mat', 's32.mat', 's26.mat', 's27.mat', 's19.mat', 's25.mat', 's31.mat', 's30.mat', 's24.mat', 's18.mat']


In [ ]:
mat_data = []
mat_target = []
for name in mat_file_list:
    print(name)
    file = scipy.io.loadmat(mat_dir_path +"/"+ name)
    tmp_data = np.delete(file['data'],[32,33,34,35,36,37,38,39], axis = 1)
#     print(tmp_data.shape)
    mat_data.extend(tmp_data)
    mat_target.extend(file['labels'])
#     print(tmp_data.shape)
#     print(file['labels'].shape)

mat_data = np.array(mat_data)
mat_target = np.array(mat_target)
print(mat_data.shape)
print(mat_target.shape)

#저장
np.save(os.path.join(os.getcwd(),'DEAP','DEAP_data_eeg'), mat_data)
np.save(os.path.join(os.getcwd(),'DEAP','DEAP_label'), mat_target)

s01.mat
s15.mat
s29.mat
s28.mat
s14.mat
s16.mat
s02.mat
s03.mat
s17.mat
s13.mat
s07.mat
s06.mat
s12.mat
s04.mat
s10.mat
s11.mat
s05.mat
s20.mat
s08.mat
s09.mat
s21.mat
s23.mat
s22.mat
s32.mat
s26.mat
s27.mat
s19.mat
s25.mat
s31.mat
s30.mat
s24.mat
s18.mat
(1280, 32, 8064)
(1280, 4)


## data load

In [ ]:
data = np.load('DEAP/DEAP_data_eeg.npy')
label = np.load('DEAP/DEAP_label.npy')
label = label[:,:2] # valence, arousal
val, aro = label[:,0], label[:,1]

print('data shape :', data.shape)
print('label shape :', label.shape)
print('val shape :', val.shape)
print('aro shape :', aro.shape)

data shape : (1280, 32, 8064)
label shape : (1280, 2)
val shape : (1280,)
aro shape : (1280,)


---

In [ ]:
# val 0,1로 변경
print(val)
val = np.array([1 if i>=5 else 0 for i in val])[:32]
print(val)

[7.71 8.1  8.58 ... 4.51 4.06 5.  ]
[1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0]


### val, aro 나누기

In [ ]:
eeg_data = data[:][:32][:]
eeg_data.shape

(32, 32, 8064)

### target = val

In [ ]:
# 훈련 세트와 검증 세트로 나누기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(eeg_data, val, stratify = val, random_state=42)

In [ ]:
# 크기 출력하기
print(train_input.shape, test_target.shape)

(24, 32, 8064) (8,)


### LSTM

In [ ]:
from tensorflow import keras
from  keras.layers import Flatten, LSTM, Dense
model = keras.Sequential()
model.add(LSTM(128,input_shape=(32,8064), dropout=0.3, return_sequences=True))
model.add(LSTM(64, dropout=0.3))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_input, train_target, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 6s 6s/step - loss: 0.9607 - accuracy: 0.5263 - val_loss: 0.4935 - val_accuracy: 0.8000
Epoch 2/10
1/1 [==============================] - 0s 85ms/step - loss: 0.8218 - accuracy: 0.5263 - val_loss: 0.4919 - val_accuracy: 0.8000
Epoch 3/10
1/1 [==============================] - 0s 78ms/step - loss: 0.7245 - accuracy: 0.5263 - val_loss: 0.4899 - val_accuracy: 0.8000
Epoch 4/10
1/1 [==============================] - 0s 87ms/step - loss: 0.6642 - accuracy: 0.5263 - val_loss: 0.4871 - val_accuracy: 0.8000
Epoch 5/10
1/1 [==============================] - 0s 83ms/step - loss: 0.6000 - accuracy: 0.5263 - val_loss: 0.4844 - val_accuracy: 0.8000
Epoch 6/10
1/1 [==============================] - 0s 92ms/step - loss: 0.5307 - accuracy: 0.5263 - val_loss: 0.4825 - val_accuracy: 0.8000
Epoch 7/10
1/1 [==============================] - 0s 80ms/step - loss: 0.4765 - accuracy: 0.5263 - val_loss: 0.4819 - val_accuracy: 0.8000
Epoch 8/10
1/1 [=============

In [ ]:
model.evaluate(test_input, test_target)
model.predict(test_input)

1/1 [==============================] - 1s 860ms/step


array([[0.68955666],
       [0.72345823],
       [0.7982828 ],
       [0.7220266 ],
       [0.8330263 ],
       [0.80665326],
       [0.79421085],
       [0.75802565]], dtype=float32)